In [ ]:
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
import math
import time
import cv2
import os

from scipy.interpolate import interp1d
# from celluloid import Camera # getting the camera
from itertools import permutations
from IPython.display import HTML
from glob import glob

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### loadBTdata.py

In [ ]:
def loadBT(dir):
    
    dfs = pd.DataFrame()
    filenames = [file
                 for path, subdir, files in os.walk(dir)
                 for file in glob(os.path.join(path, "*.txt"))]
    
#     print(filenames)
    
    if len(filenames)==0:
        print("No data in " + dir)
        
    for fname in filenames:

        print(fname)
        pi_name = fname.split('/')[5][2:5]
        data = pd.read_csv(fname,sep=',',header=None, names=["Time","ID","RSSI"])
        data["PI"] = pi_name
        dfs = pd.concat([dfs,data],axis=0, ignore_index=True)
    
    dfs = dfs.reset_index(drop=True)
    dfs_sorted = dfs.sort_values(dfs.columns[0], ascending = True)
    dfs_sorted = dfs_sorted.reset_index(drop=True)
        
    return dfs_sorted

### DistanceMeasures.py

In [ ]:
def computeDistance(startPoint, endPoint):
    return np.sqrt(((startPoint[0]-endPoint[0])**2)+((startPoint[1]-endPoint[1])**2))

def getSideFromRadius(Radius):
    
#     scale_factor = 1144/33.4772
    scale_factor = (955 - 130)/33.4772
    # 2m is the approximated distance of waist to drop ceiling for all participants.
    # 1144/33.4772 was emprically found to convert distances in real world to pixel distances.
    # It is function of input image/map resoloution.
    
    ceilingToWaist = 2 * scale_factor
     
    return int(np.sqrt(np.abs((Radius**2)-(ceilingToWaist**2))))

### EP6Partitions.py

In [ ]:
def getRoom(Loc):
    
    x=Loc[0]
    y=Loc[1]
    
    if 350 < x < 430 and 310 < y < 920:
        room = 'Activity Studio'
        
    elif 620 < x < 670 and 200 < y < 950:
        room = 'LC'
        
    elif 1060 < x < 1260 and 200 < y < 950:
        room = 'RC'    
        
    elif 600 < x < 1210 and 130 < y < 200:
        room = 'Kitchen' 
        
    elif 700 < x < 1100 and 680 < y < 800:
        room='Lounge'
        
    elif 1400 < x < 1480 and 320 < y < 730:
        room='Staff Zone'  
 
    else:
        room='Transition Zones'
        
    return room

### LocateMe.py

In [ ]:
def locator(BLE_DATA,sTime,eTime):
        
    import datetime
    # You need to pass a csv file into the form of Time (timestamp form), ID, RSSi and PI value into the locator function
    # The locator function returns a dataframe with location, rooms and the time at that location

    BLE_DATA = pd.read_csv(BLE_DATA + '.csv')
    # The selected hyeprparameters are as follows based on our grid search optimization technique

    # startTime = BLE_DATA['Time'].tolist()[0]  
    # endTime = BLE_DATA['Time'].tolist()[-1] 

    startTime = datetime.datetime(int(date.split("/")[0]), int(date.split("/")[1]), int(date.split("/")[2]), int(sTime.split(":")[0]), int(sTime.split(":")[1]), int(sTime.split(":")[2])).timestamp()  
    endTime = datetime.datetime(int(date.split("/")[0]), int(date.split("/")[1]), int(date.split("/")[2]), int(eTime.split(":")[0]), int(eTime.split(":")[1]), int(eTime.split(":")[2])).timestamp() 
    
    
    
    
    directory = '/opt/scratchspace/SSAGHAF/'
    # Loading Pi X and Y Locations on Map
    locations = np.loadtxt('PiLocations.csv', delimiter = ',', dtype = {'names': ('Pi', 'X', 'Y'),'formats': ('i','i','i')}, skiprows = 1)
    Locations = {}
    
    for location in locations:
        Locations[location[0]] = (location[1], location[2])
        

    tframe = 1         # 5s was selected as the window length ########################################################################
    RHSI_METHOD = 1    # RHSI-Edge == 1 and RHSI-Agg == 0    ########################################################################
    SS = 1             # Slide = 1 and Step = 2              ########################################################################
    
    nextp = startTime + tframe
    sTime = startTime
    location = []
    Time_loc = []
    rooms = []
    path_x = []
    path_y = []
    HITPI_IDs = []
    NUM_HITS = []
    
    NPT = []
    NHT = []
    
    scale_factor = (955 - 130)/33.4772
    
    while nextp <= endTime:
        
        temp = BLE_DATA[(BLE_DATA['Time'] >= sTime) & (BLE_DATA['Time'] <= nextp)]

        # Choose the moving type to be slide(1)/step(2)
        
        # The best method based on the hyperparameter tuning was the step method so we can put SS = 2
        # Also the best weight set was S6 which is provided below
        
        W = [0.6, 0.4, 0.0]
        w1, w2, w3  = W[0], W[1], W[2]
        
        if SS == 1:
        
            sTime = sTime + 1
            nextp = sTime + tframe
            
        else:

            sTime = sTime + tframe
            nextp = nextp + tframe
            
        
        numberOfCircles = 0
        radii = []
        RADIUSINM = []
        
        HitPiIDs,NumOfHits = np.unique(temp['PI'],return_counts=True)
       
#         print(HitPiIDs)
#         len(HitPiIDs)
        
        N = 3.5   # N = 3.5 is a function of environment (Hyperparameter)   
    
        NP = len(HitPiIDs) # number of Pi's
        
        if NP == 0:
            NH = 0 # max number of hits
        else:
            NH = np.max(NumOfHits)
        
        for PiID in HitPiIDs:
            
            RSSI = temp[temp['PI']==PiID].RSSI.mean()
            radiusInM = 10 ** ((-73 - RSSI)/(10 * N)) # You hard code the M1RSSI into -73dbm
            
            if radiusInM > 10:
                radiusInM = 10
                
            RADIUSINM.append(radiusInM)
            radii.append(getSideFromRadius(int(radiusInM * scale_factor))) 
                    
        radii = np.array(radii)
        
        
        if len(HitPiIDs) == 0:

            center_coordinates = (np.nan, np.nan) #(1160,405) modified to nan - it does not make sense to initialize randomly when there is no information
        
        if len(HitPiIDs) == 1:
            
            Pi = int(PiID)
            X, Y = Locations[Pi]
            center_coordinates = (int(X * 1830/2432), int(Y * 1167/1632))
            radius = radii[0]
        
        if len(HitPiIDs) > 1:

            perm = permutations(range(0,len(HitPiIDs)), 2)
            perm = list(perm)
            estimatedLocation = np.zeros((len(perm),2))
            weight = np.zeros(len(perm))
            
            
            # Method 1 -- RHSI-Agg: Method number: 0
            
            if RHSI_METHOD == 0:
            
                for n in range(0,len(perm)):

                    Pi1 = int(HitPiIDs[perm[n][0]])
                    Pi2 = int(HitPiIDs[perm[n][1]])
                    X1, Y1 = Locations[Pi1][0] * 1830/2432, Locations[Pi1][1] * 1167/1632
                    X2, Y2 = Locations[Pi2][0] * 1830/2432, Locations[Pi2][1] * 1167/1632
                    r1 = radii[perm[n][0]]
                    r2 = radii[perm[n][1]]

                    estimatedLocation[n,:] = [int(X1+(r1/(r1+r2))*(X2-X1)), int(Y1+(r1/(r1+r2))*(Y2-Y1))] 
                    weight[n] = NumOfHits[perm[n][0]] + NumOfHits[perm[n][1]]

                weighted_loc = estimatedLocation * weight[:,None]
                weighted_sum_x = np.sum(weighted_loc[:,0]) / np.sum(weight)
                weighted_sum_y = np.sum(weighted_loc[:,1]) / np.sum(weight)
                center_coordinates = (int(weighted_sum_x), int(weighted_sum_y))
                radius = int(1.5 * scale_factor)


            # Method 2 -- RHSI-Edge: Method number: 1
            else:
    
                for n in range(0,len(perm)):

                    Pi1 = int(HitPiIDs[perm[n][0]])
                    Pi2 = int(HitPiIDs[perm[n][1]])
                    X1, Y1 = Locations[Pi1][0] * 1830/2432, Locations[Pi1][1] * 1167/1632
                    X2, Y2 = Locations[Pi2][0] * 1830/2432, Locations[Pi2][1] * 1167/1632

                    weight[n] = NumOfHits[perm[n][0]] + NumOfHits[perm[n][1]]
                    r1 = getSideFromRadius(int(RADIUSINM[perm[n][0]] * NumOfHits[perm[n][0]]/weight[n] * scale_factor))
                    r2 = getSideFromRadius(int(RADIUSINM[perm[n][1]] * NumOfHits[perm[n][1]]/weight[n] * scale_factor))    


                    estimatedLocation[n,:] = [int(X1+(r1/(r1+r2))*(X2-X1)), int(Y1+(r1/(r1+r2))*(Y2-Y1))] 

                weighted_loc = estimatedLocation
                weighted_sum_x = np.sum(weighted_loc[:,0]) / len(weighted_loc[:,0])
                weighted_sum_y = np.sum(weighted_loc[:,1]) / len(weighted_loc[:,1])
                center_coordinates = (int(weighted_sum_x), int(weighted_sum_y))
                radius = int(1.5 * scale_factor)
            
            
        if np.isnan(center_coordinates[0]) == False and np.isnan(center_coordinates[1]) == False:
            
            if len(location) >= 2:
                
                if computeDistance((location[-1][0], location[-1][1]), (location[-2][0], location[-2][1]))!=0:
                    
                    x_MA = w1*center_coordinates[0] + w2*location[-1][0] + w3*location[-2][0]
                    y_MA = w1*center_coordinates[1] + w2*location[-1][1] + w3*location[-2][1]
                    center_coordinates = (x_MA, y_MA)
            
            
                
            location.append(center_coordinates)
            rooms.append(getRoom(center_coordinates))
            Time_loc.append(int(nextp))
            HITPI_IDs.append(HitPiIDs)
            NUM_HITS.append(NumOfHits)

        else:

            if len(location) >= 2:

                location.append(location[-1])
                rooms.append(rooms[-1])
                Time_loc.append(Time_loc[-1] + 1)
                HITPI_IDs.append(HITPI_IDs[-1])
                NUM_HITS.append(NUM_HITS[-1])

            
            
        data = {'location': location, 'rooms': rooms, 'time': Time_loc, 'PI': HITPI_IDs, '#Hits': NUM_HITS}
        dataframe = pd.DataFrame(data)

    return dataframe

## Step II

In [ ]:
BLE_DATA_SORTED = pd.read_csv('BLE_DATA_SORTED.csv')
BLE_DATA_SORTED

In [ ]:
# sTime = "17:47:39"
# eTime = "17:48:59"

date = '2023/12/8'
sTime = "17:47:00"
eTime = "17:49:00"

dataframe = locator('BLE_DATA_SORTED', sTime,eTime)

In [ ]:
dfp = dataframe.copy()
dfp

In [ ]:
directory = '/opt/scratchspace/SSAGHAF/'
# Loading Pi X and Y Locations on Map
locations = np.loadtxt('PiLocations.csv', delimiter = ',', dtype = {'names': ('Pi', 'X', 'Y'),'formats': ('i','i','i')}, skiprows = 1)
Locations = {}

Pi, X, Y = [], [], []

for location in locations:
    Pi.append(location[0])
    X.append(location[1])
    Y.append(location[2])
    
    
data = {'Pi': Pi, 'X': X, 'Y': Y}
df = pd.DataFrame(data)



img = cv2.imread("ep6_floorplan_measured_half_gridded_1_meter.jpg", cv2.IMREAD_COLOR)
img = cv2.resize(img, (610*3, 389*3))




# fig = plt.figure(figsize=(20, 10))
# plt.imshow(img)


wid = img.shape[1]
hgt = img.shape[0]

print(str(wid) + "x" + str(hgt))



color=[0,0,255] 

for i in range(0, len(df)):
    
    image = cv2.circle(img, (int(df['X'][i] * 1830/2432), int(df['Y'][i] * 1167/1632)), 10, color , -1)
#     image = cv2.putText(image, str(df['Pi'][i]), (int(df['X'][i] * 1830/2432) + 10, int(df['Y'][i] * 1167/1632) + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2, cv2.LINE_AA)

plt.figure(figsize=(20,10))
plt.imshow(image)

In [ ]:
# ## RC real data

color=[255, 0, 0] 

X, Y, T = [], [], []

for i in range(0, 15):
    
    image = cv2.circle(image, (1170, 930 - i*50), 10, color , -1)
    X.append(1170)
    Y.append(925 - i*52)
    T.append(5 + i*5)

plt.figure(figsize=(20,10))
plt.imshow(image)

Ground_Truth = {'X': X, 'Y': Y, 'T': T}
df_GT = pd.DataFrame(Ground_Truth)
df_GT

In [ ]:
# from datetime import datetime, timedelta
# import pytz  # You may need to install this module using: pip install pytz

# def convert_gmt_to_est(gmt_time_str):
#     # Parse the input string as a datetime object in GMT
#     gmt_time = datetime.strptime(gmt_time_str, '%Y-%m-%d %H:%M:%S')
#     gmt_time = pytz.utc.localize(gmt_time)

#     # Define the time zone for Eastern Standard Time (EST)
#     est_timezone = pytz.timezone('US/Eastern')

#     # Convert GMT time to EST
#     est_time = gmt_time.astimezone(est_timezone)

#     return est_time.strftime('%Y-%m-%d %H:%M:%S %Z')

In [ ]:
# # Example usage
# gmt_time_str = '2023-12-08 22:29:28'
# est_time_str = convert_gmt_to_est(gmt_time_str)
# print(f'GMT Time: {gmt_time_str}')
# print(f'EST Time: {est_time_str}')

In [ ]:
# # date = '2022/04/22'
# date = '2023/12/8'

# # # time interval and participant Mac ID -------------------------------------------------- LC
# # sTime = "16:21:40"
# # eTime = "16:30:50"

# # # Mac: "ef:d4:e6:78:1b:80"  #Green - not working
# # # Mac: "de:28:8e:55:25:67"  #Red - not working
# # Mac = "fe:a1:de:0b:9d:a7"  #Blue


# # # # time interval and participant Mac ID -------------------------------------------------- RC
# sTime = "17:30:00"
# eTime = "17:33:00"

# # Mac = "e6:80:b8:03:0b:2e" # Soheil
# Mac = "e7:fe:51:ec:25:c4" # Barun

# # # # time interval and participant Mac ID -------------------------------------------------- Kitchen
# # sTime = "16:08:20"
# # eTime = "16:17:50"

# # # Mac: "ef:d4:e6:78:1b:80"  #Red - not working
# # # Mac: "de:28:8e:55:25:67"  #Green - not working
# # Mac = "fe:a1:de:0b:9d:a7"  #Blue


# # # time interval and participant Mac ID ---------------- X - 180p ---------------------------------- Lobby
# # sTime = "16:36:00"
# # eTime = "16:38:30"

# # # Mac = "de:28:8e:55:25:67"  # Red
# # Mac = "ef:d4:e6:78:1b:80"  # Green
# # # Mac = "fe:a1:de:0b:9d:a7"  # Blue


# # # Activity Area Participants -------------------------  X + 50p ------------------------- Activity Area
# # sTime = "15:54:30"
# # eTime = "16:03:30"


# # Mac = "fe:a1:de:0b:9d:a7" # Blue
# # # Mac = "ef:d4:e6:78:1b:80" # Red
# # # Mac = "de:28:8e:55:25:67" # green

# wSize = 10

In [ ]:
# location, rooms, Time_loc, NPT, NHT, Unique_IDs = locator(date,sTime,eTime,Mac,wSize)
# extractedData = {'location': location, 'rooms': rooms, 'Pis': NPT, 'Hits': NHT, 'time': Time_loc}
# df = pd.DataFrame(extractedData)
# df

In [ ]:
dfp

In [ ]:
plt.hist(dfp['rooms'])

In [ ]:
color=[255,0,0] 

X, Y , R, T, NP, NH = [], [], [], [], [], []

for i in range(0,len(dfp['location'])):    

    X.append(dfp['location'][i][0])
    Y.append(dfp['location'][i][1]) 
    R.append(dfp['rooms'][i])
    T.append(dfp['time'][i])
            
data = {'X': X, 'Y': Y, 'R': R, 'T': T}
dfm = pd.DataFrame(data)  
time_list = dfm['T'] - dfm['T'][0]
dfm['T'] = np.array(time_list)
dfm

In [ ]:
# X, Y , R, T, NP, NH = [], [], [], [], [], []

# for i in range(0,len(dfm)-1):
#     if computeDistance((dfm['X'][i], dfm['Y'][i]), (dfm['X'][i+1], dfm['Y'][i+1]))!=0:

#         X.append(dfm['X'][i])
#         Y.append(dfm['Y'][i])
#         R.append(dfm['R'][i])
#         T.append(dfm['T'][i])
#         # NP.append(dfm['Pis'][i])
#         # NH.append(dfm['Hits'][i])

# X.append(dfm['X'][len(dfm)-1])
# Y.append(dfm['Y'][len(dfm)-1])
# R.append(dfm['R'][len(dfm)-1])
# T.append(dfm['T'][len(dfm)-1])
        
# # data = {'X': X, 'Y': Y, 'R': R, 'T': T, 'Pis': NP, 'Hits': NH}
# data = {'X': X, 'Y': Y, 'R': R, 'T': T}
# dfm = pd.DataFrame(data)

# T = dfm['T'].diff()
# T[0] = 0

# # data = {'X': X, 'Y': Y, 'R': R, 'T': T, 'Pis': NP, 'Hits': NH}
# data = {'X': X, 'Y': Y, 'R': R, 'T': T}
# dfm = pd.DataFrame(data)

# dfm['T'] = dfm['T'].cumsum()
# dfm

In [ ]:
np.floor(len(dfm)/len(df_GT))

In [ ]:
np.ceil(len(df_GT)/len(dfm))

In [ ]:
XA, YA, T = [], [], []

inner_counter = 0

for i in range(1,len(df_GT)):

    if len(dfm)>=len(df_GT):

        dynamic_interval = np.floor(len(dfm)/len(df_GT))

        print(dynamic_interval)
        dfm_reduced = dfm[int((i-1)*dynamic_interval):int(i*dynamic_interval)]
        print(dfm_reduced)
    
        if len(dfm_reduced) != 0:
            XA.append(dfm_reduced['X'].mean())
            YA.append(dfm_reduced['Y'].mean())
        else:
            XA.append(XA[-1])
            YA.append(YA[-1])
            
        T.append(i*dynamic_interval)
       
    else:

        dynamic_interval = np.ceil(len(df_GT)/len(dfm))
        
        counter = 0
    
        while counter < dynamic_interval:
            
            XA.append(dfm['X'][inner_counter])
            YA.append(dfm['Y'][inner_counter])
            T.append(dfm['T'][inner_counter])
            counter = counter + 1

        inner_counter = inner_counter + 1
        
        print(inner_counter)
        if inner_counter >= len(dfm):
            break
        

if len(dfm)>=len(df_GT):
    
    print(dfm.iloc[-2:])
    XA.append(dfm['X'].iloc[-2:].mean())
    YA.append(dfm['Y'].iloc[-2:].mean())
    T.append((i+1)*dynamic_interval)

else:
    print('dfm legnth was smaller')


        
data = {'X': XA, 'Y': YA, 'T': T}
dfp = pd.DataFrame(data)
dfp

# Check the accuracy of the estimated data

In [ ]:
dfm = dfp.copy()

if len(dfm) > len(df_GT):

    XA, YA, T = [], [], []

    dynamic_interval = np.floor(len(dfm)/len(df_GT))
    
    for i in range(1,len(df_GT)):
    
        print(i)
        dfm_reduced = dfm[int((i-1)*dynamic_interval):int(i*dynamic_interval)]
        print(dfm_reduced)
    
        if len(dfm_reduced) != 0:
            XA.append(dfm_reduced['X'].mean())
            YA.append(dfm_reduced['Y'].mean())
        else:
            XA.append(XA[-1])
            YA.append(YA[-1])
            
        T.append(i*dynamic_interval)
    
    print(dfm.iloc[-2:])
    XA.append(dfm['X'].iloc[-2:].mean())
    YA.append(dfm['Y'].iloc[-2:].mean())
    T.append((i+1)*dynamic_interval)
    
    
    data = {'X': XA, 'Y': YA, 'T': T}
    dfp = pd.DataFrame(data)
    dfp

else:

    print('len dfp was not greater than len dfm')

In [ ]:
color = [255, 165, 0]

for i in range(0,len(dfp)-1):

    image_new = cv2.circle(image, (int(dfp['X'][i]),int(dfp['Y'][i])), 10, color, -1)
    # image_new = cv2.arrowedLine(image_new, (int(dfm['X'][i]),int(dfm['Y'][i])), (int(dfm['X'][i+1]),int(dfm['Y'][i+1])), color, 2,tipLength = 0.001)
#     image_new = cv2.putText(img, str(i), (int(dfm['X'][i]), int(dfm['Y'][i])), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)


plt.figure(figsize=(20, 10))
plt.imshow(image_new)

In [ ]:
Err_RG = []

for i in range(0, len(df_GT)):
    Err_RG.append(computeDistance((dfp['X'][i], dfp['Y'][i]), (df_GT['X'][i], df_GT['Y'][i])))

Err_RG_scalded = [x * 33.4772/(955 - 130) for x in Err_RG] # Change it into the meter
AveErr = np.mean(Err_RG_scalded)
StdErr = np.std(Err_RG_scalded)
print(AveErr)

In [ ]:
Err_RG_scalded

In [ ]:
# X_G = df_GT['X'].tolist()
# Y_G = df_GT['Y'].tolist()
# T_G = df_GT['T'].tolist()

# X_P = dfp['X'].tolist()
# Y_P = dfp['Y'].tolist()
# T_P = dfp['T'].tolist()

# data = {'XG': X_G, 'YG': Y_G, 'XP': X_P, 'YP': Y_P, 'Err': Err_RG_scalded,'TG': T_G, 'TP': T_P}
# dft = pd.DataFrame(data)
# dft

In [ ]:
X_G = df_GT['X'].tolist()
Y_G = df_GT['Y'].tolist()

X_P = dfp['X'].tolist()
Y_P = dfp['Y'].tolist()

data = {'XG': X_G, 'YG': Y_G, 'XP': X_P, 'YP': Y_P, 'Err': Err_RG_scalded}
dft = pd.DataFrame(data)
dft

In [ ]:
# dft1 = dft['Err'].tolist()
# dft2 = dft['Err'].tolist()
# dft3 = dft['Err'].tolist()
# dft4 = dft['Err'].tolist()
# dft5 = dft['Err'].tolist()
# dft6 = dft['Err'].tolist()
# dft7 = dft['Err'].tolist()

In [ ]:
import pandas as pd

# Create a dictionary with the data
data = {
    'dft1': dft1,
    'dft2': dft2,
    'dft3': dft3,
    'dft4': dft4,
    'dft5': dft5,
    'dft6': dft6,
    'dft7': dft7
}

# Convert the dictionary into a DataFrame
df = pd.DataFrame(data)

In [ ]:
# Save the DataFrame to a pickle file
# df.to_pickle('locationwise_error.pkl')

# Save the DataFrame to a pickle file
# df.to_pickle('locationwise_error_one_by_one.pkl')

In [ ]:
df_ind = pd.read_pickle('locationwise_error_one_by_one.pkl')
df = df_ind

In [ ]:
df

In [ ]:
# Optionally, you can load it back like this to verify
df_total = pd.read_pickle('locationwise_error_one_by_one.pkl')
df = df_total


# Plot each column as a line
plt.figure(figsize=(10, 6))

for column in df.columns:
    plt.plot(df[column], label=column)

# Add labels and a legend
plt.xlabel('Index')
plt.ylabel('Values')
plt.title('Line Plot of dft Columns')
plt.legend()

# Show the plot
plt.show()

In [ ]:
# Optionally, you can load it back like this to verify
df_total = pd.read_pickle('locationwise_error.pkl')
df = df_total


# Plot each column as a line
plt.figure(figsize=(10, 6))

for column in df.columns:
    plt.plot(df[column][::-1].reset_index(drop=True), '-o', label=column)

# Add labels and a legend
plt.xlabel('Index')
plt.ylabel('Values')
plt.title('Line Plot of dft Columns')
plt.legend()

# Show the plot
plt.show()


In [ ]:
df.columns = [f'BLE-{i}' for i in range(1, 8)]
df

In [ ]:
# Set a modern style
sns.set_theme(style="whitegrid")

# Create the plot
plt.figure(figsize=(12, 4))

# Define region boundaries
high_density_range = range(0, 8)  # First 7 indices
low_density_range = range(8, 15)  # Last 7 indices

# Add shaded regions
plt.axvspan(0, 7, color='lightblue', alpha=0.3, label='High-Density Region')
plt.axvspan(7, 14, color='lightgreen', alpha=0.3, label='Low-Density Region')

# Plot the lines
for column in df.columns:
    plt.plot(df[column][::-1].reset_index(drop=True), 
             '-o', 
             label=column, 
             linewidth=2, 
             markersize=6)

# Add labels, title, and legend
plt.xlabel('Locations Along The Right Corridor', fontsize=14)
plt.ylabel('Localization Error (Meters)', fontsize=14)
plt.title('', fontsize=16)

# Add a legend for both lines and regions
plt.legend(title='', fontsize=12, title_fontsize=14, bbox_to_anchor=(1.05, 1), loc='upper left')

# Annotate the regions
plt.text(3, max(df.max()) * 0.9, 'High Density', fontsize=18, color='blue', ha='center', alpha=0.8)
plt.text(10.5, max(df.max()) * 0.9, 'Low Density', fontsize=18, color='green', ha='center', alpha=0.8)

# Adjust layout and grid
plt.tight_layout()
plt.grid(True, linestyle='--', alpha=0.8)

plt.savefig('error_trends_by_region.png', dpi=300, bbox_inches='tight')

# Show the plot
plt.show()



In [ ]:
dft1_reversed = df_total['dft1'][::-1].reset_index(drop=True)
dft7_reversed = df_total['dft7'][::-1].reset_index(drop=True)

plt.plot(dft1_reversed, color = 'r', linewidth = 2)
plt.plot(dft7_reversed, color = 'g', linewidth = 2)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load dataframes
df1_total = pd.read_pickle('locationwise_error_one_by_one.pkl')
df2_total = pd.read_pickle('locationwise_error.pkl')

# Assume both dataframes have the same number of columns
df1 = df1_total
df2 = df2_total

# Create subplots with as many rows as there are columns
num_columns = len(df1.columns)
fig, axs = plt.subplots(num_columns, 1, figsize=(8, 3 * num_columns))  # One column, as many rows as the columns of df1

# Plot corresponding columns of both dataframes on the same subplot
for i, column in enumerate(df1.columns):
    # Plot df1's columns individually, cumulatively from the first to the current column
    for col in df1.columns[:i+1]:
        axs[i].plot(df1[col], '-or', linewidth = 1, label=f'df1 {col}')
        
    # Plot the current column of df2
    axs[i].plot(df2[column], '-og', linewidth = 2, label=f'df2 {column}')
    
    axs[i].set_xlabel('Index')
    axs[i].set_ylabel('Values')
    axs[i].set_title(f'Plot of Columns Up to: {column}')
    
    # Move legend outside the plot
    axs[i].legend(loc='upper left', bbox_to_anchor=(1, 1))

# Adjust layout to make space for the legends outside
plt.tight_layout(rect=[0, 0, 0.85, 1])  # Leave space on the right for the legends

# Show the plot
plt.show()


In [ ]:
# Optionally, you can load it back like this to verify
df_total = pd.read_pickle('locationwise_error_one_by_one.pkl')
df = df_total


# Z-score normalization
# df_normalized_zscore = (df - df.mean()) / df.std()
# df = df_normalized_zscore

# df_normalized_minmax = (df - df.min()) / (df.max() - df.min())
# df = df_normalized_minmax


# Create a heatmap
plt.figure(figsize=(16, 5))

# Normalize the color by setting vmin and vmax
sns.heatmap(df, annot=True, cmap='YlOrBr', vmin=df.min().min(), vmax=df.max().max())

# Add a title
plt.title('Heatmap of dft Columns')

# Show the plot
plt.show()

In [ ]:
# Optionally, you can load it back like this to verify
df_total = pd.read_pickle('locationwise_error.pkl')
df = df_total


# Z-score normalization
# df_normalized_zscore = (df - df.mean()) / df.std()
# df = df_normalized_zscore

# df_normalized_minmax = (df - df.min()) / (df.max() - df.min())
# df = df_normalized_minmax


# Create a heatmap
plt.figure(figsize=(16, 5))

# Normalize the color by setting vmin and vmax
sns.heatmap(df, annot=True, cmap='YlOrBr', vmin=df.min().min(), vmax=df.max().max())

# Add a title
plt.title('Heatmap of dft Columns')

# Show the plot
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the DataFrame (already done)
df_total = pd.read_pickle('locationwise_error.pkl')
df = df_total
# Rename columns
df.rename(columns={'dft1': '#BLE-1', 'dft2': '#BLE-2', 'dft3': '#BLE-3', 'dft4': '#BLE-4', 'dft5': '#BLE-5', 'dft6': '#BLE-6', 'dft7': '#BLE-7'}, inplace=True)

# Calculate statistics
df_stats = pd.DataFrame({
    'Mean': df.mean(),
    'Std': df.std(),
    'Median': df.median(),
    'Max': df.max(),
    'Min': df.min()
})

cmap = plt.cm.copper_r  # Define colormap

# First figure: Heatmap of the data
fig1, ax1 = plt.subplots(figsize=(20, 5))
sns.heatmap(df, annot=True, cmap=cmap, vmin=df.min().min(), vmax=df.max().max(), ax=ax1)

# Reverse the y-axis and set y-ticks to start from 1
ax1.invert_yaxis()  # Reverse the y-axis
ax1.set_yticks(range(1, len(df.index) + 1))  # Set y-ticks to start from 1
ax1.set_yticklabels(range(1, len(df.index) + 1))  # Label y-ticks from 1 to N

ax1.set_title('Localization Error (m2) - Estimation Versus Ground Truth')
plt.tight_layout()

plt.savefig('heatmap_of_df_columns.png')  # Save as PNG
plt.show()


# Set up the figure and axes
fig2, ax_array = plt.subplots(1, 5, figsize=(20, 4))

# Plot each statistic in its own subplot
for i, (stat, values) in enumerate(df_stats.items()):
    # Normalize values for color mapping
    norm = plt.Normalize(vmin=values.min(), vmax=values.max())
    colors = cmap(norm(values))  # Map colors based on values

    # Plot each bar with color mapping
    bars = ax_array[i].bar(values.index, values, color=colors)
    ax_array[i].set_title(stat)
    ax_array[i].set_ylabel('Err (m)')
    ax_array[i].set_xlabel(' ')

    # Overlay dot and line plots
    ax_array[i].plot(values.index, values, marker='o', color='black', linestyle='-', linewidth=2, markersize=10)

    # Set x-ticks to vertical position
    ax_array[i].set_xticklabels(values.index, rotation=90)  # Set rotation to 90 degrees

# Tight layout and display
plt.tight_layout()

plt.savefig('statistics_bar_plots.png')  # Save as PNG
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the DataFrame (already done)
df_total = pd.read_pickle('locationwise_error.pkl')
df = df_total
# Rename columns
df.rename(columns={'dft1': 'BLE-1', 'dft2': 'BLE-2', 'dft3': 'BLE-3', 'dft4': 'BLE-4', 'dft5': 'BLE-5', 'dft6': 'BLE-6', 'dft7': 'BLE-7'}, inplace=True)

# Calculate statistics
df_stats = pd.DataFrame({
    'Mean': df.mean(),
    'Std': df.std(),
    'Median': df.median(),
    'Max': df.max(),
    'Min': df.min()
})

cmap = plt.cm.copper_r  # Define colormap

# Adjust font sizes globally
plt.rcParams.update({'font.size': 16})  # General font size for all elements

# First figure: Heatmap of the data
fig1, ax1 = plt.subplots(figsize=(20, 8))
sns.heatmap(df, annot=True, cmap=cmap, vmin=df.min().min(), vmax=df.max().max(), ax=ax1, annot_kws={'fontsize': 16})  # Increase annotation font size

# Reverse the y-axis and set y-ticks to start from 1
ax1.invert_yaxis()  # Reverse the y-axis
ax1.set_yticks(range(1, len(df.index) + 1))  # Set y-ticks to start from 1
ax1.set_yticklabels(range(1, len(df.index) + 1), fontsize=16, rotation=0)  # Make y-tick labels horizontal

ax1.set_title('Localization Error (m2) - Estimation Versus Ground Truth', fontsize=16)  # Increase title font size
ax1.set_xlabel(' ', fontsize=16)  # Increase x-axis label font size
ax1.set_ylabel('Error', fontsize=16)  # Increase y-axis label font size

plt.tight_layout()
plt.savefig('heatmap_of_df_columns.png')  # Save as PNG
plt.show()

# Set up the figure and axes for statistics bar plots
fig2, ax_array = plt.subplots(1, 5, figsize=(20, 4))

# Plot each statistic in its own subplot
for i, (stat, values) in enumerate(df_stats.items()):
    # Normalize values for color mapping
    norm = plt.Normalize(vmin=values.min(), vmax=values.max())
    colors = cmap(norm(values))  # Map colors based on values

    # Plot each bar with color mapping
    bars = ax_array[i].bar(values.index, values, color=colors)
    ax_array[i].set_title(stat, fontsize=16)  # Increase subplot title font size
    ax_array[i].set_ylabel('Err (m)', fontsize=16)  # Increase y-axis label font size
    ax_array[i].set_xlabel(' ', fontsize=16)  # Increase x-axis label font size

    # Overlay dot and line plots
    ax_array[i].plot(values.index, values, marker='o', color='black', linestyle='-', linewidth=2, markersize=10)

    # Set x-ticks to vertical position and increase font size
    ax_array[i].set_xticklabels(values.index, rotation=90, fontsize=16)  # Set rotation and font size for x-tick labels

# Tight layout and display
plt.tight_layout()
plt.savefig('statistics_bar_plots.png')  # Save as PNG
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the DataFrame (already done)
df_total = pd.read_pickle('locationwise_error.pkl')
df = df_total

# Calculate statistics
df_stats = pd.DataFrame({
    'Mean': df.mean(),
    'Std Dev': df.std(),
    'Median': df.median(),
    'Max': df.max(),
    'Min': df.min()
})

# Reverse DataFrame and set new index starting from 1
df_reversed = df.iloc[::-1]
df_reversed.index = range(1, len(df_reversed) + 1)  # Set new index starting from 1

# First figure: Heatmap of the data (reversed)
fig1, ax1 = plt.subplots(figsize=(20, 5))
sns.heatmap(df_reversed, annot=True, cmap='coolwarm', vmin=df.min().min(), vmax=df.max().max(), ax=ax1)

# Reverse the y-axis labels
ax1.set_yticks(range(1, len(df_reversed) + 1))  # Set y-ticks to start from 1
ax1.set_yticklabels(range(len(df_reversed), 0, -1))  # Reverse labels from top to bottom

ax1.set_title('Localization Error (m2) - Estimation Versus Ground Truth')
plt.tight_layout()
plt.show()

# Set up the figure and axes
fig2, ax_array = plt.subplots(1, 5, figsize=(20, 4))
cmap = plt.cm.coolwarm  # Define colormap

# Plot each statistic in its own subplot
for i, (stat, values) in enumerate(df_stats.items()):
    # Normalize values for color mapping
    norm = plt.Normalize(vmin=values.min(), vmax=values.max())
    colors = cmap(norm(values))  # Map colors based on values

    # Plot each bar with color mapping
    bars = ax_array[i].bar(values.index, values, color=colors)
    ax_array[i].set_title(stat)
    ax_array[i].set_ylabel('Err (m)')
    ax_array[i].set_xlabel(' ')

    # Overlay dot and line plots
    ax_array[i].plot(values.index, values, marker='o', color='black', linestyle='-', linewidth=2, markersize=10)

# Adjust spacing between heatmap and bar plot
plt.subplots_adjust(hspace=0.2)  # Decrease hspace to make it shorter

# Tight layout and display
plt.tight_layout()
plt.show()



# # Set up the figure and axes
# fig2, ax_array = plt.subplots(1, 5, figsize=(20, 4))
# cmap = plt.cm.coolwarm  # Define colormap

# # Determine global min and max for normalization
# global_min = min(df_stats[stat].min() for stat in df_stats)
# global_max = max(df_stats[stat].max() for stat in df_stats)

# # Plot each statistic in its own subplot
# for i, (stat, values) in enumerate(df_stats.items()):
#     # Normalize values for color mapping
#     norm = plt.Normalize(vmin=global_min, vmax=global_max)
#     colors = cmap(norm(values))  # Map colors based on values

#     # Plot each bar with color mapping
#     bars = ax_array[i].bar(values.index, values, color=colors)
#     ax_array[i].set_title(stat)
#     ax_array[i].set_ylabel('Err (m)')
#     ax_array[i].set_xlabel(' ')

#     # Overlay dot and line plots
#     ax_array[i].plot(values.index, values, marker='o', color='black', linestyle='-', linewidth=2, markersize=10)

#     # Set y-axis limits for the last subplot
#     if i == 4:  # Last subplot (index 4)
#         ax_array[i].set_ylim(0, 2)  # Set y-axis limits to 0 to 6

# # Adjust spacing between heatmap and bar plot
# plt.subplots_adjust(hspace=0.2)  # Decrease hspace to make it shorter

# # Tight layout and display
# plt.tight_layout()
# plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Set the figure size (width, height in inches)
plt.figure(figsize=(6, 3))  # Example: 10 inches wide, 6 inches tall

# Plot the 'Err' values
plt.plot(dft['Err'], '-ok', linewidth = 2, label='Error')

# Calculate the average of 'Err'
average_err = dft['Err'].mean()

# Plot the average line
plt.axhline(y=average_err, color='r', linestyle='--', linewidth = 2, label=f'Average: {average_err:.2f}')

# Add labels and a legend
plt.xlabel('Index')
plt.ylabel('Error')
plt.title('Error Plot with Average Line')
plt.legend()

# Display the plot
plt.show()

## Visualization

In [ ]:
AVG_BLE_1 = [9.09, 7.59, 5.71, 5.37, 4.38, 4.13, 2.99]
AVG_BLE_2 = [5.14, 4.46, 5.61, 4.97, 3.89, 2.89, np.nan]
AVG_BLE_3 = [3.58, 3.92, 5.28, 4.99, 2.94, np.nan, np.nan]
AVG_BLE_4 = [3.18, 3.74, 5.29, 3.17, np.nan, np.nan, np.nan]
AVG_BLE_5 = [3.40, 3.75, 3.88, np.nan, np.nan, np.nan, np.nan]
AVG_BLE_6 = [3.41, 2.90, np.nan, np.nan, np.nan, np.nan, np.nan]
AVG_BLE_7 = [2.72, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]

In [ ]:
AVG = np.array([[AVG_BLE_1, AVG_BLE_2, AVG_BLE_3, AVG_BLE_4, AVG_BLE_5, AVG_BLE_6, AVG_BLE_7]])
AVG = np.transpose(AVG)
AVG

In [ ]:
plt.figure(figsize=(6, 3))

lw, ms = 1, 5
x = list(range(1, 8))
plt.plot(x, AVG[0][0], '-ok', markersize = ms, linewidth = lw, label='# of BLE = 1')
plt.plot(x, AVG[0][1], '-or', markersize = ms, linewidth = lw, label='# of BLE = 2')
plt.plot(x, AVG[0][2], '-ob', markersize = ms, linewidth = lw, label='# of BLE = 3')
plt.plot(x, AVG[0][3], '-om', markersize = ms, linewidth = lw, label='# of BLE = 4')
plt.plot(x, AVG[0][4], '-oy', markersize = ms, linewidth = lw, label='# of BLE = 5')
plt.plot(x, AVG[0][5], '-oc', markersize = ms, linewidth = lw, label='# of BLE = 6')
plt.plot(x, AVG[0][6], '-og', markersize = ms, linewidth = lw, label='# of BLE = 7')
plt.legend(loc='upper right')

plt.xlabel('Total Number of Cases')
plt.ylabel('Error')

In [ ]:
plt.figure(figsize=(4, 2))

lw, ms = 1, 5
x = list(range(1, 8))
# plt.legend(loc='upper right')

# plt.plot(x, AVG[0], '-ok', markersize = ms, linewidth = lw)
# plt.plot(x, AVG[1], '-or', markersize = ms, linewidth = lw)
# plt.plot(x, AVG[2], '-ob', markersize = ms, linewidth = lw)
# plt.plot(x, AVG[3], '-om', markersize = ms, linewidth = lw)
# plt.plot(x, AVG[4], '-oy', markersize = ms, linewidth = lw)
plt.plot(x, AVG[5], '-oc', markersize = ms, linewidth = lw)
# plt.plot(x , AVG[6], '-og', markersize = ms, linewidth = lw)

plt.xlim([0.8, 7.2])
plt.ylim([2., 10])
plt.xlabel('Number of BLE Devices')
plt.ylabel('Error')

In [ ]:
data = [
    [9.09, 7.59, 5.71, 5.37, 4.38, 4.13, 2.99],
    [5.14, 4.46, 5.61, 4.97, 3.89, 2.89, np.nan],
    [3.58, 3.92, 5.28, 4.99, 2.94, np.nan, np.nan],
    [3.18, 3.74, 5.29, 3.17, np.nan, np.nan, np.nan],
    [3.40, 3.75, 3.88, np.nan, np.nan, np.nan, np.nan],
    [3.41, 2.90, np.nan, np.nan, np.nan, np.nan, np.nan],
    [2.72, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define the data
data = [
    [9.09, 7.59, 5.71, 5.37, 4.38, 4.13, 2.99],
    [5.14, 4.46, 5.61, 4.97, 3.89, 2.89, np.nan],
    [3.58, 3.92, 5.28, 4.99, 2.94, np.nan, np.nan],
    [3.18, 3.74, 5.29, 3.17, np.nan, np.nan, np.nan],
    [3.40, 3.75, 3.88, np.nan, np.nan, np.nan, np.nan],
    [3.41, 2.90, np.nan, np.nan, np.nan, np.nan, np.nan],
    [2.72, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
]

# Extract the first column and handle NaN values
first_column = [row[0] for row in data]

# Define the number of BLE devices (1 to 7 for this example)
ble_devices = list(range(1, len(first_column) + 1))

# Create a figure with a larger size
plt.figure(figsize=(8, 3))

# Add horizontal and vertical lines
plt.axhline(y=3.5, color='red', linestyle='--', linewidth=3)
plt.axvline(x=3, color='green', linestyle='--', linewidth=3)

# Plot the first column with enhanced styling
plt.plot(ble_devices, first_column, marker='o', color='black', markersize=8, linestyle='-', linewidth=2, label='Error Measurement')

# Set labels and title with larger font sizes
plt.xlabel('Number of BLE Devices', fontsize=14)
plt.ylabel('Error in Meters', fontsize=14)
plt.title('Error in Meters vs Number of BLE Devices', fontsize=16)

# Set x-ticks to match the number of BLE devices
plt.xticks(ble_devices, fontsize=12)
plt.yticks(fontsize=12)

# Add gridlines for better readability
plt.grid(True, which='both', linestyle='--', linewidth=0.7, alpha=0.7)

# Set limits on the axes if necessary (adjust based on your data)
plt.ylim(0, max([val for val in first_column if not np.isnan(val)]) + 1)  # Set the y-limit based on your data

# Add a legend
plt.legend(fontsize=12)

# Show the plot
plt.tight_layout()  # Adjust layout for better

# Save the figure
plt.savefig('Error_in_Meters_vs_Number_of_BLE_Devices.png', dpi=300, bbox_inches='tight')  # Save as PNG with 300 DPI
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Define the data
data = [
    [9.09, 7.59, 5.71, 5.37, 4.38, 4.13, 2.99],
    [5.14, 4.46, 5.61, 4.97, 3.89, 2.89, np.nan],
    [3.58, 3.92, 5.28, 4.99, 2.94, np.nan, np.nan],
    [3.18, 3.74, 5.29, 3.17, np.nan, np.nan, np.nan],
    [3.40, 3.75, 3.88, np.nan, np.nan, np.nan, np.nan],
    [3.41, 2.90, np.nan, np.nan, np.nan, np.nan, np.nan],
    [2.72, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
]

# Convert the data to a numpy array
data_array = np.array(data).T

# Create a heatmap
plt.figure(figsize=(10, 6))
ax = sns.heatmap(data_array, annot=True, fmt=".2f", cmap='YlOrBr_r', 
                 cbar=True, mask=np.isnan(data_array), linewidths=2, linecolor='white')

# Hide x-axis and y-axis
ax.set_xticks([])
ax.set_yticks([])

# Optionally hide axis labels if you want no labels at all
plt.xlabel('Number of BLE devices')
plt.ylabel('Error (meters)')

# Show the plot
plt.tight_layout()
plt.savefig('Heatmap_Error_in_Meters_vs_Number_of_BLE_Devices.png', dpi=300, bbox_inches='tight')  # Save as PNG with 300 DPI
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Define the data
data = [
    [9.09, 7.59, 5.71, 5.37, 4.38, 4.13, 2.99],
    [5.14, 4.46, 5.61, 4.97, 3.89, 2.89, np.nan],
    [3.58, 3.92, 5.28, 4.99, 2.94, np.nan, np.nan],
    [3.18, 3.74, 5.29, 3.17, np.nan, np.nan, np.nan],
    [3.40, 3.75, 3.88, np.nan, np.nan, np.nan, np.nan],
    [3.41, 2.90, np.nan, np.nan, np.nan, np.nan, np.nan],
    [2.72, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
]

# Convert the data to a numpy array
data_array = np.array(data).T

# Create a heatmap
plt.figure(figsize=(10, 6))
ax = sns.heatmap(data_array, annot=True, fmt=".2f", cmap='YlOrBr_r', 
                 cbar=True, mask=np.isnan(data_array), linewidths=2, linecolor='white')

# Access the color bar and move ticks to the left
cbar = ax.collections[0].colorbar
cbar.ax.yaxis.set_ticks_position('left')  # Move ticks to the left

# Hide x-axis and y-axis
ax.set_xticks([])
ax.set_yticks([])

# Optionally hide axis labels if you want no labels at all
plt.xlabel('Number of BLE devices')
plt.ylabel('Error (meters)')

# Show the plot
plt.tight_layout()
plt.savefig('Heatmap_Error_in_Meters_vs_Number_of_BLE_Devices.png', dpi=300, bbox_inches='tight')  # Save as PNG with 300 DPI
plt.show()
